Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Data Driven Audio Signal Processing - A Tutorial with Computational Examples

Winter Semester 2022/23 (Master Course #24512)

- lecture: https://github.com/spatialaudio/data-driven-audio-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise

Feel free to contact lecturer frank.schultz@uni-rostock.de

# Gradient Descent (GD)

### Analytical Loss Function in 2D

Suppose, that the (very artificial) loss function of a model with two parameters $\beta_1$ and $\beta_2$ is analytically given as

$$\mathcal{L}(\beta_1, \beta_2) = (\beta_1 - 2)^2 + (\beta_2 - 1)^4 - (\beta_2 -1)^2$$

In this toy example there is no data dependency involved, which is not very practical, but good to understand the essence of finding a minimum numerically.

In order to find potential minima, an thereby the optimum model parameters $\beta_1$ and $\beta_2$, we need to solve gradient for zero

$$\nabla \mathcal{L} = 
\begin{bmatrix}
\frac{\partial \mathcal{L}}{\partial \beta_1}\\
\frac{\partial \mathcal{L}}{\partial \beta_2}
\end{bmatrix}=
\mathbf{0}$$

The required partial derivatives of first order are 

$$\frac{\partial \mathcal{L}}{\partial \beta_1} = 2 (\beta_1 - 2)^1$$

$$\frac{\partial \mathcal{L}}{\partial \beta_2} = 4 (\beta_2 - 1)^3 - 2(\beta_2 -1)^1$$

A check with the Hessian of $\mathcal{L}(\beta_1, \beta_2)$ yields whether we deal with a minimum, maximum, saddle point or neither of them for each of the zero gradient conditions.

We get **first minimum** at
$$\beta_{1,min1} = 2\qquad \beta_{2,min1} = 1+\frac{1}{\sqrt{2}}$$

We get **second minimum** at
$$\beta_{1,min2} = 2\qquad \beta_{2,min2} = 1-\frac{1}{\sqrt{2}}$$

Both minima yield the same function value
$$\mathcal{L}(\beta_{1,min}, \beta_{2,min}) = -\frac{1}{4},$$
so we deal actually with **two optimum models**, as there is **no global minimum** with only one lowest function value.

We have **one saddle point**, that actually separates the two minima at
$$\beta_{1,saddle} = 2\qquad \beta_{2,saddle} = 1$$

with function value
$$\mathcal{L}(\beta_{1,saddle}, \beta_{2,saddle}) = 0$$


In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

matplotlib_widget_flag = True

In [ ]:
# analytical solutions
minimum1, fminimum1 = np.array([2, 1+1/np.sqrt(2)]), -1/4
minimum2, fminimum2 = np.array([2, 1-1/np.sqrt(2)]), -1/4
saddle, fsaddle = np.array([2, 1]), 0


def get_gradient(beta):
    beta_gradient = np.array([2*(beta[0]-2),
                              4*(beta[1]-1)**3 - 2*(beta[1]-1)])
    return beta_gradient

In [ ]:
def f(x, y):
    # our loss function from above
    # only with nicer to read variables x,y
    # instead of beta1, beta2
    return (x-2)**2 + (y-1)**4 - (y-1)**2


x, y = np.linspace(0, 4, 2**7), np.linspace(-1/2, 5/2, 2**7)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

In [ ]:
col_min, col_max, no_col = -1/4, 5, 51
col_tick = np.linspace(col_min, col_max, no_col, endpoint=True)
cmap = plt.cm.magma_r
norm = mpl.colors.BoundaryNorm(col_tick, cmap.N)

if matplotlib_widget_flag:
    %matplotlib widget
fig = plt.figure()
ax = plt.axes(projection='3d')
c = ax.plot_surface(X, Y, Z,
                    rstride=1, cstride=1,
                    cmap=cmap, norm=norm,
                    edgecolor='none')
ax.plot(minimum1[0], minimum1[1], fminimum1, 'C0o')
ax.plot(minimum2[0], minimum2[1], fminimum2, 'C0o')
ax.plot(saddle[0], saddle[1], fsaddle, 'C0o')
cbar = fig.colorbar(c, ax=ax,
                    ticks=col_tick[::no_col//10],
                    label=r'$\mathcal{L}$')
ax.set_xlim(x[0], x[-1])
ax.set_ylim(y[0], y[-1])
ax.set_zlim(-1/4, 5)
ax.set_xlabel(r'$\beta_0$')
ax.set_ylabel(r'$\beta_1$')
ax.set_zlabel(r'$\mathcal{L}$')

### Finding Minimum with Gradient Descent (GD)

In [ ]:
def my_gradient_descent(beta, beta_gradient, step_size):
    beta -= beta_gradient * step_size 
    return beta

In [ ]:
def plot_contour_of_my_gradient_descent():
    fig, ax = plt.subplots()
    ax.contour(X, Y, Z, cmap='magma_r')
    ax.plot(beta_path[0], beta_path[1], 'C0.-', lw=0.3)
    ax.plot(minimum1[0], minimum1[1], 'kx', ms=10)
    ax.plot(minimum2[0], minimum2[1], 'kx', ms=10)
    ax.plot(saddle[0], saddle[1], 'kx', ms=10)
    
    ax.set_xlabel(r'$\beta_1$')
    ax.set_ylabel(r'$\beta_2$')
    ax.set_title(r'last calculated beta1 = {f1:5.4f}, beta2 = {f2:5.4f}'.format(
        f1=beta_path[0, -1], f2=beta_path[1, -1]))
    ax.axis('equal')
    ax.grid('True')

#### Gradient Descent Towards Minimum 1

The 

- chosen number of steps and
- chosen step size and
- chosen starting point 

work out to find the minimum at

$$\beta_{1,min1} = 2\qquad \beta_{2,min1} = 1+\frac{1}{\sqrt{2}}$$ 

using straightforward gradient descent (GD).

In [ ]:
steps = 2**5
beta = np.array([4, 1.075])
step_size = 1/5

beta_path = np.zeros((2, steps+1))
beta_path[:, 0] = beta  # store chosen init beta
for step in range(steps):
    # this is the GD
    beta_gradient = get_gradient(beta)
    beta = my_gradient_descent(beta, beta_gradient, step_size)
    # store our GD path
    beta_path[:, step+1] = beta
# and plot
plot_contour_of_my_gradient_descent()

#### Gradient Descent Towards Minimum 2

Chosen number of steps, chosen step size and chosen starting point work out to approach the minimum at

$$\beta_{1,min2} = 2\qquad \beta_{2,min2} = 1-\frac{1}{\sqrt{2}},$$

which we however not reached yet. Increasing number of steps and/or slightly decreasing the step size should bring us to the minimum 2. 

In [ ]:
steps = 2**6
beta = np.array([0.0, -0.5])
step_size = 1 / 40


beta_path = np.zeros((2, steps+1))
beta_path[:, 0] = beta  # store chosen init beta
for step in range(steps):
    # this is the GD
    beta_gradient = get_gradient(beta)
    beta = my_gradient_descent(beta, beta_gradient, step_size)
    # store our GD path
    beta_path[:, step+1] = beta
# and plot
plot_contour_of_my_gradient_descent()

#### Gradient Descent Towards Minimum 2 Along the Saddle Point Ridge

This GD path is interesting as we start very close to the saddle point ridge, i.e. we start with a chosen starting value $\beta_2=0.9999$, which is very close to $\beta_{2,saddle} = 1$.
The GD then initially moves along the saddle point ridge, but fortunately drops of to find its way to minimum 2.
The chosen values for number of steps and step size are much more critical here compared to above examples.

What happens if we choose `beta = np.array([4., 1.0001])`?

In [ ]:
steps = 2**6
beta = np.array([4., 0.9999])
step_size = 1 / 10


beta_path = np.zeros((2, steps+1))
beta_path[:, 0] = beta  # store chosen init beta
for step in range(steps):
    # this is the GD
    beta_gradient = get_gradient(beta)
    beta = my_gradient_descent(beta, beta_gradient, step_size)
    # store our GD path
    beta_path[:, step+1] = beta
# and plot
plot_contour_of_my_gradient_descent()

#### Gradient Descent Towards Saddle Point

In the examples above fortunately the GD approached or even finally found a minimum, which is what we desire when learn optimum model parameters.

Things could go wrong, if we set up things badly. One prominent and illustrative example is given here: we never reach one of the two minima, but rather we *die* on the saddle point. This is because the GD can by its gradient concept never drop of the ridge of the saddle point.
Here in this **toy example**, we know **that and why** this happens. What about practical, very complicated loss functions?!?! *Dying* on a saddle point is probably not a nicely learned model.

In [ ]:
steps = 2**6
beta = np.array([4., 1.])
step_size = 1 / 10

beta_path = np.zeros((2, steps+1))
beta_path[:, 0] = beta  # store chosen init beta
for step in range(steps):
    # this is the GD
    beta_gradient = get_gradient(beta)
    beta = my_gradient_descent(beta, beta_gradient, step_size)
    # store our GD path
    beta_path[:, step+1] = beta
# and plot
plot_contour_of_my_gradient_descent()

#### Gradient Descent Ends in Oscillation

Another bad thing occurs in this example: the chosen large step size of $1$ and the chosen $\beta_2=1$ on the saddle point ridge yield a GD that oscillates between

$$[\beta_1, \beta_2]^\mathrm{T} = [4, 1]^\mathrm{T}$$

$$[\beta_1, \beta_2]^\mathrm{T} = [0, 1]^\mathrm{T}$$

We should check the analytical equations, these numbers are not by accident. The gradient is

$$\frac{\partial \mathcal{L}}{\partial \beta_1} = 2 (\beta_1 - 2)^1$$

$$\frac{\partial \mathcal{L}}{\partial \beta_2} = 4 (\beta_2 - 1)^3 - 2(\beta_2 -1)^1$$

We start with initial $$\beta_{actual} = [\beta_1, \beta_2]^\mathrm{T} = [4, 1]^\mathrm{T},$$
hence
$$\frac{\partial \mathcal{L}}{\partial \beta_1}\bigg|_{\beta_{actual}} = 2 (4 - 2)^1 = 4$$
$$\frac{\partial \mathcal{L}}{\partial \beta_2}\bigg|_{\beta_{actual}} = 4 (1 - 1)^3 - 2(1 -1)^1 = 0$$
The update rule in GD is
$$\beta_{new} = \beta_{actual} - step \cdot \nabla\mathcal{L}\bigg|_{\beta_{actual}},$$
with inserted numbers
$$\beta_{new} = 
\begin{bmatrix}4\\1
\end{bmatrix}
- 1 \cdot
\begin{bmatrix}4\\0
\end{bmatrix}=
\begin{bmatrix}0\\1
\end{bmatrix}
$$
GD moves to the next step, so the model parameter vector which was newly calculated becomes the actual vector $\beta_{new} \rightarrow \beta_{actual}$.

So, we deal with $$\beta_{actual} = [\beta_1, \beta_2]^\mathrm{T} = [0, 1]^\mathrm{T},$$
hence
$$\frac{\partial \mathcal{L}}{\partial \beta_1}\bigg|_{\beta_{actual}} = 2 (0 - 2)^1 = -4$$
$$\frac{\partial \mathcal{L}}{\partial \beta_2}\bigg|_{\beta_{actual}} = 4 (1 - 1)^3 - 2(1 -1)^1 = 0$$
The update rule in GD is
$$\beta_{new} = \beta_{actual} - step \cdot \nabla\mathcal{L}\bigg|_{\beta_{actual}},$$
with inserted numbers
$$\beta_{new} = 
\begin{bmatrix}0\\1
\end{bmatrix}
- 1 \cdot
\begin{bmatrix}-4\\0
\end{bmatrix}=
\begin{bmatrix}4\\1
\end{bmatrix}
$$
In text step, we again get
$$\beta_{new} =
\begin{bmatrix}0\\1
\end{bmatrix}
$$, we see the oscillation.
In this toy example the oscillation is very obvious and easy to check by the simple equations.
In practice, oscillations could be more complex (more than two states) and hard to trace.
We should be aware of such phenomenon as a minimum is never reached.

In [ ]:
steps = 2**3
beta = np.array([4., 1])
step_size = 1


beta_path = np.zeros((2, steps+1))
beta_path[:, 0] = beta  # store chosen init beta
for step in range(steps):
    # this is the GD
    beta_gradient = get_gradient(beta)
    beta = my_gradient_descent(beta, beta_gradient, step_size)
    # store our GD path
    beta_path[:, step+1] = beta
# and plot
plot_contour_of_my_gradient_descent()
beta_path

### Gradient Descent Exploding

We have seen a **stable GD oscillation** in the **example** directly **above**. Things could get even worse.
In this example we deal with a dangerous phenomenon referred to as **exploding gradient descent** in textbooks.
Instead of approaching one minimum the GD starts to **oscillate** and with successively **climbing up** out of the valley.
In the example here we have chosen the numbers, such that double precision still can handle the numbers.
Very often such exploding gradients end in Nan, Inf for the gradients and the model parameters. Obviously, that's not useful at all.

We should be aware, that changing `step_size = 0.29` to `step_size = 0.28` the GD does not explode, but find its way to minimum 1. So, a tiny change in one number rules over success vs. failure.
This indicates the importance of being cautious against chosen parameters and actually understanding what and why algorithms do the things they do.

In [ ]:
steps = 2**3
beta = np.array([4., 2.5])

if True:
    step_size = 0.29  # GD explodes
else:
    step_size = 0.28  # GD approaches minimum1

beta_path = np.zeros((2, steps+1))
beta_path[:, 0] = beta  # store chosen init beta
for step in range(steps):
    # this is the GD
    beta_gradient = get_gradient(beta)
    print('step', step, 'gradient', beta_gradient)
    beta = my_gradient_descent(beta, beta_gradient, step_size)
    # store our GD path
    beta_path[:, step+1] = beta
# and plot
plot_contour_of_my_gradient_descent()
print('beta vector along the GD')
beta_path

#### Gradient Descent Ends in More Complex Oscillation But Does Not Explode

In this example, we deal with a more complex oscillation which however is stable, i.e. the GD does not explode.

We should simply figure out how to change the parameters `steps` and `step_size`
- make the GD explode
- make the GD find a minimum

Which minimum is found in the latter case? Why must this be the case?

In [ ]:
steps = 2**7  # default: 2**7
beta = np.array([4., 1.075])
step_size = 1  # default: 1

beta_path = np.zeros((2, steps+1))
beta_path[:, 0] = beta  # store chosen init beta
for step in range(steps):
    # this is the GD
    beta_gradient = get_gradient(beta)
    beta = my_gradient_descent(beta, beta_gradient, step_size)
    # store our GD path
    beta_path[:, step+1] = beta
# and plot
plot_contour_of_my_gradient_descent()

## Copyright

- the notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources)
- feel free to use the notebooks for your own purposes
- the text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/)
- the code of the IPython examples is licensed under the [MIT license](https://opensource.org/licenses/MIT)
- please attribute the work as follows: *Frank Schultz, Data Driven Audio Signal Processing - A Tutorial Featuring Computational Examples, University of Rostock* ideally with relevant file(s), github URL https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise, commit number and/or version tag, year.